# Материалы

- [Reuven M. Lerner - Generators, coroutines, and nanoservices || PyCon Africa 2020](https://youtu.be/tkoaeVS2zRQ)
- [Curious Course on Coroutines and Concurrency (David Beazley)](https://youtu.be/Z_OAlIhXziw)
- http://dabeaz.com/coroutines/Coroutines.pdf
- https://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-yield-from-syntax-in-python-3-3
- https://stackoverflow.com/questions/35518986/whats-the-difference-between-yield-from-and-yield-in-python-3-3-2

## Функция

In [1]:
def myfunc():
    return 1
    return 2
    return 3

In [2]:
import dis

# Even Python compiler ignores return 2 return 3
dis.dis(myfunc)  # => bytecodes from our function

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


## Генератор (generator)

`yield` turns function into `generator function`. When Python compiles your function it notices `yield` and tags the function as a `generator function`.

generator_function() => generator (object)

In [3]:
def mygen():
    yield 1
    yield 2
    yield 3

In [4]:
mygen()

<generator object mygen at 0x7f8ed17528f0>

In [5]:
dis.show_code(mygen)  # Notice the flag GENERATOR!

Name:              mygen
Filename:          /tmp/ipykernel_40355/1466075880.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 1
   2: 2
   3: 3


In [6]:
dis.dis(mygen)  # три YIELD_VALUE (нет игнорирования!)

              0 GEN_START                0

  2           2 LOAD_CONST               1 (1)
              4 YIELD_VALUE
              6 POP_TOP

  3           8 LOAD_CONST               2 (2)
             10 YIELD_VALUE
             12 POP_TOP

  4          14 LOAD_CONST               3 (3)
             16 YIELD_VALUE
             18 POP_TOP
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [7]:
## Iterator protocol

In [8]:
for i in 'abc':
    print(i)

a
b
c


In [9]:
for i in mygen():
    print(i)

1
2
3


In [10]:
g = mygen()
g

<generator object mygen at 0x7f8ed1752dc0>

In [11]:
iter(g)

<generator object mygen at 0x7f8ed1752dc0>

In [12]:
g is iter(g)

True

In [13]:
next(g), next(g), next(g)

(1, 2, 3)

In [14]:
try:
    next(g)
except StopIteration as e:
    print(type(e))

<class 'StopIteration'>


## Корутина

In [15]:
def mygen():
    x = None
    while True:
        x = yield x
        x *= 5

In [16]:
g = mygen()

In [17]:
next(g)  # priming, same as g.send(None)

In [18]:
g.send(5)

25

In [19]:
g.send(10)

50

In [20]:
g.send('abc')

'abcabcabcabcabc'

In [21]:
g.send([1, 2, 3])

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

### Walrus + yield

In [22]:
def mygen():
    x = None
    while x := (yield x):
        x *= 5

In [23]:
g = mygen()
g.send(None)
g.send(2)

10

### Coroutine as a nanoservices

In [24]:
# https://worldweather.wmo.int/en/dataguide.html
# "Russian Federation";"Moscow";"206"
import requests

class DifferentCityException(Exception):
    pass

def get_forecasts():
    while city_id := (yield 'Enter city id (Moscow id 206):'):
        weather = requests.get(f'https://worldweather.wmo.int/en/json/{city_id}_en.json').json()
        try:
            for one_forecast in weather['city']['forecast']['forecastDay']:
                yield one_forecast
        except DifferentCityException:
            continue

In [25]:
g = get_forecasts()
next(g)

'Enter city id (Moscow id 206):'

In [26]:
g.send(44)

{'forecastDate': '2022-09-21',
 'wxdesc': '',
 'weather': 'Partly Cloudy',
 'minTemp': '25',
 'maxTemp': '29',
 'minTempF': '77',
 'maxTempF': '84',
 'weatherIcon': 2202}

In [27]:
g.send(None)  # повторить

{'forecastDate': '2022-09-22',
 'wxdesc': '',
 'weather': 'Clear',
 'minTemp': '24',
 'maxTemp': '29',
 'minTempF': '75',
 'maxTempF': '84',
 'weatherIcon': 2502}

In [28]:
g.throw(DifferentCityException)

'Enter city id (Moscow id 206):'

In [29]:
g.send(206)

{'forecastDate': '2022-09-22',
 'wxdesc': '',
 'weather': 'Rain',
 'minTemp': '8',
 'maxTemp': '14',
 'minTempF': '46',
 'maxTempF': '57',
 'weatherIcon': 1401}

In [30]:
g.send(None)

{'forecastDate': '2022-09-23',
 'wxdesc': '',
 'weather': 'Rain',
 'minTemp': '8',
 'maxTemp': '14',
 'minTempF': '46',
 'maxTempF': '57',
 'weatherIcon': 1401}

## yield from

What yield from does is it establishes a `transparent bidirectional connection` between the caller and the sub-generator:

- The connection is "transparent" in the sense that it will propagate everything correctly too, not just the elements being generated (e.g. exceptions are propagated).

- The connection is "bidirectional" in the sense that data can be both sent from and to a generator.

## Мой пример

In [31]:
from functools import wraps

# Грузовичок срывается из гаража, доезжает до первой yield-остановки,
# груз выгружает (он при этом игнорируется) и
# ждет нового груза (он будет загружен через `send`).
def prime(generator_func):
    @wraps(generator_func)
    def inner(*args, **kwargs):
        generator = generator_func(*args, **kwargs)
        next(generator)
        return generator
    return inner

In [32]:
def gen1():
    """gen1"""
    x = 'gen1 usage'
    while x := (yield x):
        x *= 5

In [33]:
def gen2():
    """gen2"""
    x = 'gen2 usage'
    while x := (yield x):
        x *= 10

In [34]:
def combined(gen_functions):
    G = {n: gf for n, gf in enumerate(gen_functions, start=1)}
    welcome = '\n'.join(f'{n}: {gf.__doc__}' for n, gf in G.items())
    while n := (yield welcome):
        if n in G:
            yield from G[n]()
        continue

In [35]:
g = combined([gen1, gen2])

In [36]:
g.send(None)

'1: gen1\n2: gen2'

In [37]:
g.send(1)

'gen1 usage'

In [38]:
g.send(20)

100

In [39]:
g.send(None)

'1: gen1\n2: gen2'

In [40]:
g.send(2)

'gen2 usage'

In [41]:
g.send(2)

20

In [42]:
g.send('')

'1: gen1\n2: gen2'

In [43]:
g.send(100)

'1: gen1\n2: gen2'

## Type hinting

https://mypy.readthedocs.io/en/stable/kinds_of_types.html#generators